In [1]:
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

device = torch.device('cpu')

TRAIN_DATA_PATH = "./data/movies_metadata.csv"


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
data = pd.read_csv(TRAIN_DATA_PATH)
data = data.head(2500)

/var/folders/gq/xnxpqv5x7jz19nnym4s4_l940000gn/T/ipykernel_3414/975283158.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(TRAIN_DATA_PATH)


In [3]:
data['overview'] = data['overview'].fillna('')
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,25471,tt0115669,zh,变脸,Wang Bianlian is an aging street performer kno...,...,1997-12-20,1.0,91.0,"[{'iso_639_1': 'zh', 'name': '普通话'}]",Released,NaN,The King of Masks,False,7.8,12.0
2496,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10769, 'n...",NaN,4154,tt0138874,en,Three Seasons,Three Seasons wants to show individuals triump...,...,1999-04-30,0.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The war is over. But a personal battle contin...,Three Seasons,False,6.8,7.0
2497,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,28519,tt0155388,en,The Winslow Boy,Early 20th century England: while toasting his...,...,1999-04-16,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Winslow Boy,False,6.4,22.0
2498,False,NaN,1453000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,3309,tt0037913,en,Mildred Pierce,"After her cheating husband leaves her, Mildred...",...,1945-09-24,5638000.0,111.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A mother's love leads to murder,Mildred Pierce,False,7.4,81.0


In [4]:
tokens = []
for i in tqdm(range(len(data))):
    token = tokenizer(data.iloc[i]['original_title'], return_tensors="pt", padding=True, truncation=True)
    tokens.append(token)
tokens = np.array(tokens)

100%|██████████| 2500/2500 [00:00<00:00, 8118.89it/s]


In [5]:
original_titles = data['original_title'].tolist()
original_titles_tokenized = tokenizer(original_titles, padding=True, truncation=True, return_tensors="pt").to(device)

In [6]:
input_ids = original_titles_tokenized['input_ids'][:, 0]
attention_mask = original_titles_tokenized['attention_mask'][:, 0]
token_type_ids = original_titles_tokenized['token_type_ids'][:, 0]

print(f"""input_ids shape {input_ids.shape}
attention_mask shape {attention_mask.shape}
token_type_ids shape {token_type_ids.shape}""")

input_ids shape torch.Size([2500])
attention_mask shape torch.Size([2500])
token_type_ids shape torch.Size([2500])
